In [1]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

In [2]:
with sqlite3.connect('../input/database.sqlite') as con:

    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)

# Overview of the data

In [3]:
countries.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [4]:
leagues.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [5]:
leagues.shape

(11, 3)

In [6]:
matches.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

# Parse out the possession in matches

In [7]:
matches['possession'].dtype

dtype('O')

In [8]:
matches.possession.describe()

count              14217
unique              8420
top       <possession />
freq                5798
Name: possession, dtype: object

looks like the data in this field is as an xml object. thank you to willinghorse for showing me how to convert the xml tags in these fields 

In [9]:
import xml.etree.ElementTree as ET

In [10]:
# get all tags in xml
def getTags(xMLcolumnNm):
    elemList = []
    # iterate each row in the XML column
    for index, row in matches[pd.notnull(matches[xMLcolumnNm])].iterrows():
        # read in xml data
        tree = ET.ElementTree(ET.fromstring(row[xMLcolumnNm]))
        for elem in tree.iter():
            elemList.append(elem.tag) # append tag name

    # remove duplicates of tag names
    return list(set(elemList))

In [11]:
# parse all information from XML column
def parseXMLData(xMLcolumnNm):
    tags = getTags(xMLcolumnNm) # get a list of all tags
       
    tagLists = {} # host all other tags
    pos = []
    otherList = {'match_id':[], 'pos_x':[], 'pos_y':[]} # host match id
    
    for tag in tags:
        tagLists[tag] = [] # initiate tag lists   
        
    for index, row in matches[pd.notnull(matches[xMLcolumnNm])].iterrows():
        game_id = row['match_api_id'] # this helps identify match
        # rea-in XML data
        tree = ET.ElementTree(ET.fromstring(row[xMLcolumnNm]))
        root = tree.getroot()  
        
        for event in root.findall('value'):
            otherList['match_id'].append(game_id)
            for tag in tags:
                if(event.find(tag) is None):
                    tagLists[tag].append(None)
                else:
                    tagLists[tag].append(event.find(tag).text) 
                    
            # get position information
            if(event.find('coordinates') is None): 
                pos.append(None)
                pos.append(None)
            else:  
                for value in event.findall("coordinates/value"):
                    pos.append(value.text)
                    
    otherList['pos_y'] = pos[1::2]  # Elements from list1 starting from 1 iterating by 2
    otherList['pos_x'] = pos[0::2]  # Elements from list1 starting from 0 iterating by 2
            
    xmlInfo = {**otherList, **tagLists}
    return pd.DataFrame(xmlInfo)

In [13]:
possession_detail = parseXMLData('possession')

In [14]:
shoton_detail = parseXMLData('shoton')

In [15]:
print(type(possession_detail))
print(possession_detail.shape)

<class 'pandas.core.frame.DataFrame'>
(34815, 21)


In [16]:
"we have possession data for " + str(possession_detail.match_id.nunique()) + " games"

'we have possession data for 8419 games'

In [17]:
print(type(shoton_detail))
print(shoton_detail.shape)

<class 'pandas.core.frame.DataFrame'>
(93755, 21)


In [18]:
"we have possession data for " + str(shoton_detail.match_id.nunique()) + " games"

'we have possession data for 8463 games'

## Possession data merging

In [31]:
possession_detail.columns

Index(['awaypos', 'card_type', 'comment', 'del', 'elapsed', 'elapsed_plus',
       'event_incident_typefk', 'goal_type', 'homepos', 'id', 'injury_time',
       'match_id', 'n', 'pos_x', 'pos_y', 'possession', 'sortorder', 'stats',
       'subtype', 'type', 'value'],
      dtype='object')

In [38]:
possession_detail['homepos'] = possession_detail['homepos'].astype(float)
possession_detail['awaypos'] = possession_detail['awaypos'].astype(float)
possession_detail['elapsed'] = possession_detail['elapsed'].astype(float)

In [39]:
possession_detail['flag'] = possession_detail['elapsed'].apply(lambda x : '1h' if x <= 45 else '2h')

In [40]:
possession_fields = ['match_id','elapsed','homepos','awaypos','flag']

In [41]:
possession_detail = possession_detail[possession_fields]

In [42]:
possession_detail.dtypes

match_id      int64
elapsed     float64
homepos     float64
awaypos     float64
flag         object
dtype: object

In [43]:
pd = possession_detail.groupby('match_id').agg({'homepos' : 'mean','awaypos' : 'mean'})
pd = pd.reset_index()

## Shots on target merging

In [22]:
shoton_detail.columns

Index(['blocked', 'card_type', 'coordinates', 'del', 'elapsed', 'elapsed_plus',
       'event_incident_typefk', 'goal_type', 'id', 'match_id', 'n', 'player1',
       'pos_x', 'pos_y', 'shoton', 'sortorder', 'stats', 'subtype', 'team',
       'type', 'value'],
      dtype='object')

In [24]:
shoton_detail.head()

,blocked,card_type,coordinates,del,elapsed,elapsed_plus,event_incident_typefk,goal_type,id,match_id,...,player1,pos_x,pos_y,shoton,sortorder,stats,subtype,team,type,value
0,None,None,None,None,3,None,61,None,378828,489042,...,24154,None,None,None,0,None,blocked_shot,10260,shoton,None
1,None,None,None,None,7,None,154,None,378866,489042,...,24157,None,None,None,2,None,header,10260,shoton,None
2,None,None,None,None,14,None,153,None,378922,489042,...,30829,None,None,None,1,None,shot,10260,shoton,None
3,None,None,None,None,14,None,153,None,378923,489042,...,30373,None,None,None,2,None,shot,10260,shoton,None
4,None,None,None,None,17,None,137,None,378951,489042,...,30373,None,None,None,3,None,distance,10260,shoton,None


## Merge the data

### take the relevant columns out of the match data

In [25]:
matches.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

In [26]:
matchcols = ['match_api_id', 'home_team_api_id', 'away_team_api_id','home_team_goal','away_team_goal','date']
m_df = matches[matchcols]

### join in the team information

In [27]:
teams.columns

Index(['id', 'team_api_id', 'team_fifa_api_id', 'team_long_name',
       'team_short_name'],
      dtype='object')

In [28]:
m_df = m_df.merge(teams[['team_api_id','team_long_name']],left_on = 'home_team_api_id', right_on = 'team_api_id')
m_df = m_df.merge(teams[['team_api_id','team_long_name']],left_on = 'away_team_api_id', right_on = 'team_api_id')

In [29]:
m_df.columns

Index(['match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'date', 'team_api_id_x',
       'team_long_name_x', 'team_api_id_y', 'team_long_name_y'],
      dtype='object')

### layer in the possession detail

In [44]:
mpd = m_df.merge(pd,left_on = 'match_api_id',right_on = 'match_id')

In [45]:
mpd

,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,date,team_api_id_x,team_long_name_x,team_api_id_y,team_long_name_y,match_id,awaypos,homepos
0,489042,10260,10261,1,1,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,489042,45.250000,54.750000
1,839802,10260,10261,3,0,2010-08-16 00:00:00,10260,Manchester United,10261,Newcastle United,839802,37.750000,62.250000
2,1024845,10260,10261,1,1,2011-11-26 00:00:00,10260,Manchester United,10261,Newcastle United,1024845,42.500000,57.500000
3,1229300,10260,10261,4,3,2012-12-26 00:00:00,10260,Manchester United,10261,Newcastle United,1229300,44.666667,55.333333
4,1474518,10260,10261,0,1,2013-12-07 00:00:00,10260,Manchester United,10261,Newcastle United,1474518,52.750000,47.250000
5,1724158,10260,10261,3,1,2014-12-26 00:00:00,10260,Manchester United,10261,Newcastle United,1724158,33.000000,67.000000
6,1988734,10260,10261,0,0,2015-08-22 00:00:00,10260,Manchester United,10261,Newcastle United,1988734,35.000000,65.000000
7,489063,9825,10261,3,0,2008-08-30 00:00:00,9825,Arsenal,10261,Newcastle United,489063,40.500000,59.500000
8,839915,9825,10261,0,1,2010-11-07 00:00:00,9825,Arsenal,10261,Newcastle United,839915,49.000000,51.000000
9,1025678,9825,10261,2,1,2012-03-12 00:00:00,9825,Arsenal,10261,Newcastle United,1025678,41.250000,58.750000


## Begin the analysis

In [ ]:
matches_df['goal_diff'] = matches_df['home_team_goal'] - matches_df['away_team_goal']
matches_df['poss_diff'] = matches_df['homepos_y'].astype(float) - matches_df['awaypos_y'].astype(float)

In [ ]:
matches_df['goal_diff'].describe()

In [ ]:
matches_df['poss_diff'].describe()

In [ ]:
from scipy.stats import pearsonr
x = matches_df['goal_diff']
y = matches_df['poss_diff']

pearsonr(x,y)

In [ ]:
#still a few nulls that we need to get rid of

In [ ]:
a = pd.concat([x,y],axis = 1)
nullindex = a[a['poss_diff'].isnull() == True].index.values
a = a.drop(nullindex)
a.to_csv('test.csv')

In [ ]:
x = a['goal_diff']
y = a['poss_diff']

pearsonr(x,y)

In [ ]:
pearsonr(x,y)[0]**2